# Исследовательский проект по НИС «Анализ данных в Python»
<div style="text-align: right">
    Проект подготовлен студентами БПИ228
    <br>
    Лысиным Кириллом и Гакал Анжеликой
</div>

## Часть 1.

## Часть 2.

In [33]:
#import requests
import datetime
from selenium import webdriver
import pandas as pd
import time
from bs4 import BeautifulSoup
import os.path


    
driver = webdriver.Chrome()

with open('visited_links.txt', 'w+') as f:
    visited_links = set(f.readlines())


def get_item_info(url):
    # Можно сделать словарь вместо массива для более удобного обращения к элементам
    item_info = []
    info_dict = {}
    
    # Тут должны вызываться методы, возвращающие строки с нужными значениями, которые будут добавляться в item_info
    soup = get_soup(url)
    if(soup == None):
        return
    
    general_info = str(soup.findAll('meta', {'property':"og:description"})[0])

    #через список
    item_info.append(get_item_name(general_info))
    item_info.append(get_item_price(general_info))
    item_info.append(get_item_category(general_info))
    item_info.append(get_item_region(soup))
    item_info.append(get_item_placement_time(soup))
    item_info.append(get_seller_name(soup))
    item_info.append(get_number_of_ads(soup))
    item_info.append(get_seller_rating(soup))
    item_info.append(get_number_of_views(soup))
    item_info.append(get_number_of_images(soup))
    item_info.append(url)
    
    #через словарь
    info_dict['Name'] = get_item_name(general_info)
    info_dict['Price (in rubles)'] = get_item_price(general_info)
    info_dict['Category'] = get_item_category(general_info)
    info_dict['Region'] = get_item_region(soup)
    info_dict['Time/date of placement'] = get_item_placement_time(soup)
    info_dict['Seller\'s name'] = get_seller_name(soup)
    info_dict['Number of seller\'s ads'] = get_number_of_ads(soup)
    info_dict['Seller rating'] = get_seller_rating(soup)
    info_dict['Number of views'] = get_number_of_views(soup)
    info_dict['Number of images'] = get_number_of_images(soup)
    info_dict['Link'] = url
    
    return item_info, info_dict

def get_soup(url, counter = 0):

    time.sleep(1)
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        return soup
    # Если не удалось сразу получить страницу, то программа пробует еще несколько раз
    except:
        if(counter < 3):
            return get_soup(url, counter + 1)
    # Если после несольких попыток так и не удалось считать содержимое страницы, то этот метод вернет None

def get_item_name(info):
    try:
        name = info[15:info.find('–') - 1]
        return name
    except:
        return None

def get_item_price(info):
    if not('Цена: ' in info):
        return 0
    price = float(info[info.find('Цена: ') + 6:info.find('руб.') - 1].replace(' ', ''))
    return price

def get_item_category(info):
    try:
        category = info[info.find('раздела «') + 9:info.find('».')]
        return category
    except:
        return None

def get_item_region(soup):
    try:
        region = list(soup.findAll('span', class_='sc-cOxWqc hNYaaO'))[0].text
        return region
    except:
        return None

def get_item_placement_time(soup):
    try:
        s = list(soup.findAll('dd', class_='sc-cOxWqc sc-fVmuvm eOPaPs dVayGV'))[-1].text
        now = datetime.datetime.now()
        if('Сегодня в' in s):
            return datetime.datetime(now.year, now.month, now.day, int(s[-5:-3]), int(s[-2:]))
        if('Вчера в' in s):
            return (datetime.datetime(now.year, now.month, now.day, int(s[-5:-3]), int(s[-2:])) + datetime.timedelta(days=-1))
        if('Позавчера в' in s):
            return (datetime.datetime(now.year, now.month, now.day, int(s[-5:-3]), int(s[-2:])) + datetime.timedelta(days=-2))


        months = ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
        for j in range(len(months)):
            if(months[j].lower() in s):
                return datetime.datetime(now.year, j + 1, int(s[:2]), 12)
        return datetime.datetime(day=int(s[:2]), month=int(s[3:5]), year=int(s[6:10]), hour = 12)
    except:
        return None

def get_seller_name(soup):
    try:
        name_of_seller = list(soup.findAll('span', {'data-test-component':"UserNameClick"}))[0].text
        name_of_seller = name_of_seller[:name_of_seller.find('(') - 1]
        return name_of_seller
    except:
        return None

def get_number_of_ads(soup):
    try:
        n = list(soup.findAll('span', {'data-test-component':"UserNameClick"}))[0].text
        n = int(n[n.find('(')+1:n.find(' объявл')])
        return n
    except:
        return None

def get_seller_rating(soup):
    try:
        rating = soup.find('span', class_="sc-cOxWqc eDoIYl")
        return float(rating.text.replace(',', '.')) if rating != None else None
    except:
        return None

def get_number_of_views(soup):
    try:
        number_of_views = list(soup.findAll('dd', class_='sc-cOxWqc sc-fVmuvm eOPaPs dVayGV'))
        return int(number_of_views[-2].text)
    except:
        return None

def get_number_of_images(soup):
    try:
        images = list(soup.findAll('img', class_="sc-fBnnfK"))
        return len(images)
    except:
        return None

def get_links(scroll_count):
    
    url = r'https://youla.ru/'
    driver.get(url)
    time.sleep(1)
    
    for i in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    links = [str(i.a) for i in soup.findAll('span', class_="sc-llGDqb sc-gqgnwQ fEAASo hZGRky")]
    links = [r'https://youla.ru' + s[s.find(r'href="') + 6 : s.find(r'" rel')] for s in links]
    return links



n = 100
for i in range(n):
    items = []
    print(f"Processing batch {i + 1} out of {n}:", end='')
    links = get_links(1)
    for i in range(len(links)):
        if(int(i * 100/len(links))//5 > int((i - 1) * 100/len(links))//5):
            print(' .', end='')
        link = links[i]
        if(link in visited_links):
            continue
        items.append(get_item_info(link)[1])
        visited_links.add(link)
    print(' ✓')

    df = pd.DataFrame(items)
    if(len(df) > 0):
        df = df.set_index('Name')
        if(os.path.isfile('items.xlsx')):
            prev_df = pd.read_excel('items.xlsx', index_col = 0)
            df = pd.concat([prev_df, df])

        df.to_excel('items.xlsx')

print("Done! The data has been saved to the file items.xlsx")

with open('visited_links.txt', 'a') as f:
    for line in visited_links:
        f.write(f"{line}\n")



driver.quit()
df

Processing batch 1 out of 100: . . . . . . . . . . . . . . . . . . . ✓
Processing batch 2 out of 100: . . . . . . . . . . . . . . . . . . . ✓
Processing batch 3 out of 100: . . . . . . . . . . . . . . . . . . . ✓
Processing batch 4 out of 100: . . . . . . . . . . . . . . . . . . . ✓
Processing batch 5 out of 100: . . . . . . . . . . . . . . . . . . . ✓
Processing batch 6 out of 100: . . . . . . . . . . . . . . . . . . . ✓
Processing batch 7 out of 100: . . . . . . . . . . . . . . . . . . . ✓
Processing batch 8 out of 100: . . . . . . . . . . . . . . . . . . . ✓
Processing batch 9 out of 100: . . . . . . . . . . . . . . . . . . . ✓
Processing batch 10 out of 100: . . . . . . . . . . . . . . . . . . . ✓
Processing batch 11 out of 100: . . . . . . . . . . . . . . . . . . . ✓
Processing batch 12 out of 100: . . . . . . . . . . . . . . . . . . . ✓
Processing batch 13 out of 100: . . . . . . . . . . . . . . . . . . . ✓
Processing batch 14 out of 100: . . . . . . . . . . . . . . . . . . . ✓
P

,Price (in rubles),Category,Region,Time/date of placement,Seller's name,Number of seller's ads,Seller rating,Number of views,Number of images,Link
Name,,,,,,,,,,
Пальто,0.0,Женский гардероб,Фрязино,2023-01-22 21:47:00,Екатерина О.,8.0,5.0,220.0,2,https://youla.ru/all/zhenskaya-odezhda/verhnya...
Щенки Кангал Карабаш (анатолийская овчарка),0.0,Животные,Люберцы,2023-01-22 21:47:00,Александр Г.,1.0,NaN,257.0,6,https://youla.ru/all/zhivotnye/sobaki/shchienk...
Ботинки,0.0,Детский гардероб,Мытищи,2023-01-22 21:47:00,Марина Ш.,161.0,4.9,6.0,3,https://youla.ru/mytischi/detskaya-odezhda/obu...
Новые трусы на мальчика,0.0,Детский гардероб,Балашиха,2023-01-22 21:47:00,Светлана Т.,20.0,5.0,80.0,0,https://youla.ru/all/detskaya-odezhda/nizhnee-...
Щенки в добрые руки в дар,0.0,Животные,Мытищи,2023-01-22 21:47:00,Людмила Сидорова,21.0,5.0,44.0,7,https://youla.ru/mytischi/zhivotnye/sobaki/shc...
...,...,...,...,...,...,...,...,...,...,...
Санки,2000.0,Детские товары,Москва,2023-01-17 12:00:00,Продавец П.,1.0,5.0,9.0,8,https://youla.ru/all/detskie/drugoe/sanki-63bd...
Стиральная машинка 6кг Samsung,4000.0,Бытовая техника,Москва,2023-01-21 20:02:00,Нина М.,5.0,NaN,110.0,3,https://youla.ru/moskva/bytovaya-tekhnika/stir...
Шторы/ фотошторы,1500.0,Для дома и дачи,Голицыно,2023-01-18 12:00:00,Виктория К.,45.0,4.9,754.0,2,https://youla.ru/golitsyno/dom-dacha/tekstil-k...


In [26]:
import datetime
def get_datetime(s):
    now = datetime.datetime.now()
    if('Сегодня в' in s):
        return datetime.datetime(now.year, now.month, now.day, int(s[-5:-3]), int(s[-2:]))
    if('Вчера в' in s):
        return (datetime.datetime(now.year, now.month, now.day, int(s[-5:-3]), int(s[-2:])) + datetime.timedelta(days=-1))
    if('Позавчера в' in s):
        return (datetime.datetime(now.year, now.month, now.day, int(s[-5:-3]), int(s[-2:])) + datetime.timedelta(days=-2))

    
    months = ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
    for j in range(len(months)):
        if(months[j].lower() in s):
            return datetime.datetime(now.year, j + 1, int(s[:2]), 12)
    return datetime.datetime(day=int(s[:2]), month=int(s[3:5]), year=int(s[6:10]), hour = 12)
print(get_datetime('Сегодня в 8:29'))

2023-01-22 08:29:00
